<a href="https://colab.research.google.com/github/arunangshudutta/EE5121_assignment_1/blob/main/Part_2_c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import cvxpy as cp

In [2]:
# filter response
u = np.array([1, 0.8, -0.2, 0.5, 0])
# Topelitz matrix
A = np.zeros((5, 5))
for i in range(5):
    for j in range(5):
        if i >= j:
            A[i, j] = u[i - j]

B = np.array([
    [ 1,   -1,    0,   0,   1],
    [ 2.8, -0.8,  1,  -1,   0.8],
    [ 2.4,  1.2,  1.8, -1.8, -1.2],
    [ 2.9, -1.7,  0.6, -0.6,  1.7],
    [ 0.4, -1.8, -0.7,  0.7,  1.8]
])

print("Matrix A")
print(A)
print("Matrix B")
print(B)


Matrix A
[[ 1.   0.   0.   0.   0. ]
 [ 0.8  1.   0.   0.   0. ]
 [-0.2  0.8  1.   0.   0. ]
 [ 0.5 -0.2  0.8  1.   0. ]
 [ 0.   0.5 -0.2  0.8  1. ]]
Matrix B
[[ 1.  -1.   0.   0.   1. ]
 [ 2.8 -0.8  1.  -1.   0.8]
 [ 2.4  1.2  1.8 -1.8 -1.2]
 [ 2.9 -1.7  0.6 -0.6  1.7]
 [ 0.4 -1.8 -0.7  0.7  1.8]]


In [4]:
print(cp.installed_solvers())

['CLARABEL', 'CVXOPT', 'GLPK', 'GLPK_MI', 'HIGHS', 'OSQP', 'SCIPY', 'SCS']


### Part 1:

In [8]:

X = cp.Variable((5, 5))
constraints = [A @ X == B]
objective = cp.Minimize(cp.normNuc(X)) # nuclear-norm minimization
prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.SCS)

print("=== Part 1: Noise-free ===")
print("Optimal value (nuclear norm):", prob.value)
print("Optimal X:\n", np.round(X.value,3))
print("Residual ||AX - B||_F:", np.linalg.norm(A @ X.value - B, 'fro'))



=== Part 1: Noise-free ===
Optimal value (nuclear norm): 7.92808264676376
Optimal X:
 [[ 1. -1.  0. -0.  1.]
 [ 2.  0.  1. -1. -0.]
 [ 1.  1.  1. -1. -1.]
 [ 2. -2. -0.  0.  2.]
 [-2. -0. -1.  1.  0.]]
Residual ||AX - B||_F: 7.623333163486916e-08


### Part 2: Noisy case

In [9]:

np.random.seed(42)
noise = np.diag(np.random.normal(0, np.sqrt(0.1), 5))
B_noisy = B + noise

eps = np.sqrt(5 * 0.1)   # reasonable tolerance (≈ expected Frobenius norm of noise)

X_noisy = cp.Variable((5, 5))
constraints_noisy = [cp.norm(A @ X_noisy - B_noisy, 'fro') <= eps]
objective_noisy = cp.Minimize(cp.normNuc(X_noisy))
prob_noisy = cp.Problem(objective_noisy, constraints_noisy)
prob_noisy.solve(solver=cp.SCS)

print("\n=== Part 2: Noisy ===")
print("Optimal value (nuclear norm):", prob_noisy.value)
print("Optimal X (noisy):\n", np.round(X_noisy.value,3))
print("Residual ||AX - B_noisy||_F:", np.linalg.norm(A @ X_noisy.value - B_noisy, 'fro'))


=== Part 2: Noisy ===
Optimal value (nuclear norm): 6.472188353795223
Optimal X (noisy):
 [[ 1.318 -1.091  0.098  0.015  1.07 ]
 [ 1.494  0.202  0.912 -0.802 -0.202]
 [ 1.491  0.636  1.152 -1.048 -0.629]
 [ 1.071 -1.52  -0.274  0.373  1.493]
 [-0.755 -0.443 -0.652  0.6    0.438]]
Residual ||AX - B_noisy||_F: 0.7071067844650878
